In [ ]:
"""libraries required"""
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install timm
!pip install torchattacks

In [ ]:
"""connect to google drive to store and retrieve data"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Generate spectrograms and save in a h5 file**

In [ ]:
from torch.utils.data import Dataset
import torch
import torchaudio
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

class ESCDataset(Dataset):
    def __init__(self, annotations_file, audio_dir, sample_rate, device):
        self.device = device
        self.sample_rate = sample_rate
        self.transform = torchvision.transforms.Compose([
            # convert signal to mel spectrogam
            torchaudio.transforms.MelSpectrogram(sample_rate=self.sample_rate, n_fft=2048, hop_length=512).to(self.device),
            # convert to log scale
            torchaudio.transforms.AmplitudeToDB().to(self.device),
            # resize to 224x224
            torchvision.transforms.Resize((224, 224), antialias=True).to(self.device),
            # change to [0, 1] range using min max scaling
            torchvision.transforms.Lambda(lambda x: (x - x.min()) / (x.max() - x.min()))
        ])
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.data = {}

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        if (self.data.get(index)):
            return self.data[index][0], self.data[index][1]

        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, _ = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)

        mel_spectrogram = self.transform(signal)

        # self.data[index] = [mel_spectrogram_db, label]
        return mel_spectrogram, label


    def _get_audio_sample_path(self, index):
        filename = self.annotations.iloc[index, 0]
        path = os.path.join(self.audio_dir, filename)
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 2]

In [ ]:
SAMPLE_RATE=44100
ANNOTATIONS_FILE = "/content/drive/MyDrive/data/ESC-50-master/ESC-50-master/meta/esc50.csv"
AUDIO_DIR = "/content/drive/MyDrive/data/ESC-50-master/ESC-50-master/audio"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using device: {device}')

esc = ESCDataset(ANNOTATIONS_FILE, AUDIO_DIR, SAMPLE_RATE, device)

print(f"Total number of samples present in dataset: {len(esc)}")

In [ ]:
import h5py

h5_file_path = '/content/drive/MyDrive/data/AdvData/ESC50.h5'
with h5py.File(h5_file_path, 'w') as hf:
    for i in range(len(esc)):
        data, label = esc[i]

        # Create datasets within the HDF5 file
        grp = hf.create_group(f'sample_{i}')
        grp.create_dataset('data', data=data.cpu().numpy())
        grp.create_dataset('label', data=label)

print("Dataset saved to", h5_file_path)

Dataset saved to /content/drive/MyDrive/data/AdvData/ESC50.h5


# Directly load data from h5 file

In [ ]:
import h5py
import torch
from torch.utils.data import TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

data_list = []
label_list = []

# Open the HDF5 file
h5_file_path = '/content/drive/MyDrive/data/AdvData/ESC50.h5'
with h5py.File(h5_file_path, 'r') as hf:
    # Iterate over the samples in the HDF5 file
    for sample_name in hf.keys():
        data = torch.tensor(hf[sample_name]['data'][:])
        label = hf[sample_name]['label'][()]

        # Append data and label to lists
        data_list.append(data)
        label_list.append(label)

esc = TensorDataset(torch.stack(data_list), torch.tensor(label_list))

**Function to train the model and calculate metrics**

In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import random_split
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt


# function to train the model
def train_model(model, dataset, learning_rate=2e-4, num_epochs=10, batch_size=128, stepSize=10, gamma=0.1, validation_split=0.2, patience=5, scheduler=None):
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    scheduler = StepLR(optimizer, step_size=stepSize, gamma=gamma)

    # Split dataset into training and validation sets
    num_samples = len(dataset)
    num_validation_samples = int(validation_split * num_samples)
    num_training_samples = num_samples - num_validation_samples
    training_dataset, validation_dataset = random_split(dataset, [num_training_samples, num_validation_samples])

    train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    best_valid_loss = float('inf')
    patience_counter = 0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        for batch_input, batch_target in train_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            optimizer.zero_grad()

            predictions = model(batch_input)

            loss = loss_fn(predictions, batch_target)
            loss.backward()

            optimizer.step()

        print(f"  Training Loss: {loss.item()}")

        # Validation phase
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            correct = 0
            total = 0
            for batch_input, batch_target in valid_loader:
                batch_input, batch_target = batch_input.to(device), batch_target.to(device)
                predictions = model(batch_input)
                _, predicted = torch.max(predictions, 1)
                total += batch_target.size(0)
                correct += (predicted == batch_target).sum().item()
                valid_loss += loss_fn(predictions, batch_target).item()

            accuracy = correct / total
            valid_loss /= len(valid_loader)
            print(f"  Validation Loss: {valid_loss}, Accuracy: {accuracy * 100:.2f}%")

            # Check for early stopping
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping after {patience} epochs of no improvement.")
                    return

        scheduler.step()

    print("Training finished.")

# function to calculate metrics
def calculate_metrics(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128, shuffle=True)
    model = model.to(device)
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input, batch_target = batch_input.to(device), batch_target.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))
            all_targets.append(batch_target)

    all_predictions = torch.cat(all_predictions).cpu().numpy()
    all_targets = torch.cat(all_targets).cpu().numpy()

    accuracy = accuracy_score(all_targets, all_predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted')
    # print(f"Precision: {precision:.4f}")
    # print(f"Recall: {recall:.4f}")
    # print(f"F1 Score: {f1:.4f}")

    # cm = confusion_matrix(all_targets, all_predictions)
    # # Plot the confusion matrix using Seaborn
    # sns.heatmap(cm, annot=True)
    # plt.title("Confusion Matrix")
    # plt.xlabel("Predicted Label")
    # plt.ylabel("True Label")
    # plt.show()

# Function for generating and storing adversarial datasets.

In [ ]:
# import torchattacks
import h5py
import time

# def generate_adversarial(dataset, model, model_name):
#     data_loader = DataLoader(dataset, batch_size=32)
#     model = model.to(device)

#     attack_dict = {}
#     attack_dict['fgsm'] = torchattacks.FGSM(model, eps=8/255)
#     attack_dict['bim'] = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
#     attack_dict['pgd'] = torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=10, random_start=True)
#     attack_dict['cw'] = torchattacks.CW(model, c=1, kappa=0, steps=50, lr=0.01)

#     for name, attk in attack_dict.items():
#         st = time.time()
#         i = 0
#         current_batch = 0
#         h5_file_path = f'/content/drive/MyDrive/data/AdvData/ESC50/{model_name}/{name}.h5'
#         with h5py.File(h5_file_path, 'a') as hf:
#             print(f"started {name}")
#             for batch_input, batch_target in data_loader:

#                 # generate adv image for image
#                 batch_input, batch_target = batch_input.to(device), batch_target.to(device)
#                 adv_images = attk(batch_input, batch_target)

#                 # write in h5 file
#                 for j in range(batch_input.shape[0]):
#                     grp_name = f'sample_{i}'
#                     if grp_name in hf:
#                         del hf[grp_name]  # Delete existing group
#                     grp = hf.create_group(f'sample_{i}')
#                     grp.create_dataset('data', data=adv_images[j].cpu().numpy())
#                     grp.create_dataset('label', data=batch_target[j].cpu())
#                     i += 1
#                 if (current_batch % 100 == 0):
#                     print(f"{current_batch} - ", end="")
#                 current_batch += 1
#             print("\n")

#         end = time.time()
#         print(f'{name} adv dataset for {model_name}')
"""
runs the given model on all adv dataset of `adv_dir` directory
"""
def calculate_accuracy_for_adv_data(model, model_name, adv_dir):
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']

    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/ESC50/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        print(f'Stats for {model_name} on {adv_dir}/{attk} dataset', end=" ")
        calculate_metrics(model, dataset_class)
        del dataset_class
        del data_list
        del label_list

def for_all(model, model_name):
    adv_dirs = ['resnet18', 'resnet50', 'vit_base', 'mixer']
    for adv in adv_dirs:
        calculate_accuracy_for_adv_data(model, model_name, adv)

# ResNet 18

In [ ]:
import timm
import torch.nn as nn

resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=50, bias=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

**Training and saving the model**

In [ ]:
torch.cuda.empty_cache()
train_model(resnet18, esc, learning_rate=1e-3, num_epochs=12, batch_size=128)

torch.save(resnet18.state_dict(), '/content/drive/MyDrive/data/Models/ESC50/resnet18.pth')

Epoch 1/12
  Training Loss: 3.5638744831085205
  Validation Loss: 3.9305001497268677, Accuracy: 0.50%
Epoch 2/12
  Training Loss: 2.943418502807617
  Validation Loss: 3.913180112838745, Accuracy: 2.25%
Epoch 3/12
  Training Loss: 2.4121944904327393
  Validation Loss: 3.8179672956466675, Accuracy: 4.00%
Epoch 4/12
  Training Loss: 1.8432769775390625
  Validation Loss: 3.584494650363922, Accuracy: 7.75%
Epoch 5/12
  Training Loss: 1.2750822305679321
  Validation Loss: 3.3709701895713806, Accuracy: 12.75%
Epoch 6/12
  Training Loss: 0.8386158347129822
  Validation Loss: 3.462745428085327, Accuracy: 16.00%
Epoch 7/12
  Training Loss: 0.5087845921516418
  Validation Loss: 1.948540449142456, Accuracy: 48.00%
Epoch 8/12
  Training Loss: 0.3709898591041565
  Validation Loss: 1.748500019311905, Accuracy: 49.00%
Epoch 9/12
  Training Loss: 0.2844506502151489
  Validation Loss: 0.976961076259613, Accuracy: 74.00%
Epoch 10/12
  Training Loss: 0.29286277294158936
  Validation Loss: 0.91079416871070

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet18.pth', map_location=torch.device(device))
resnet18.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
generate_adversarial(esc, resnet18, 'resnet18')

all attacks loaded
dataset for bim generated
bim adv dataset for resnet18 stored, time 39.967392921447754
dataset for pgd generated
pgd adv dataset for resnet18 stored, time 40.63640093803406
dataset for cw generated
cw adv dataset for resnet18 stored, time 134.08330655097961


In [ ]:
for_all(resnet18, 'resnet18')

In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'resnet18'


    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/ESC50/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)

        resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
        resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet18.fc = nn.Linear(in_features=512, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet18.pth')
        resnet18.load_state_dict(checkpoint)
        r18 = ret_pred(resnet18, dataset_class)

        print(f'Ratio for {adv_dir}/{attk} dataset')

        resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
        resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet50.fc = nn.Linear(in_features=2048, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')
        resnet50.load_state_dict(checkpoint)
        r50 = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
        vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        vit_base.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')
        vit_base.load_state_dict(checkpoint)
        vb = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
        mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        mixer.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/mixer.pth')
        mixer.load_state_dict(checkpoint)
        m = ret_pred(mixer, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        del dataset_class
        del data_list
        del label_list

In [ ]:
cal_ratio()

Ratio for resnet18/fgsm dataset
0.5675 0.076 0.3565
0.7965 0.0175 0.186
0.921 0.0145 0.0645
Ratio for resnet18/bim dataset
0.8385 0.0285 0.133
0.7965 0.0245 0.179
0.9255 0.017 0.0575
Ratio for resnet18/pgd dataset
0.847 0.0365 0.1165
0.7985 0.025 0.1765
0.9245 0.0155 0.06
Ratio for resnet18/cw dataset
0.954 0.0185 0.0275
0.803 0.034 0.163
0.9235 0.0155 0.061


# ResNet 50

In [ ]:
import timm

resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet50.fc = nn.Linear(in_features=2048, out_features=50, bias=True)

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(resnet50, esc, learning_rate=1e-3, num_epochs=12, batch_size=64)

torch.save(resnet50.state_dict(), '/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')

Epoch 1/12
  Training Loss: 3.3976597785949707
  Validation Loss: 3.8907803807939803, Accuracy: 1.50%
Epoch 2/12
  Training Loss: 2.4365880489349365
  Validation Loss: 3.692789043698992, Accuracy: 7.50%
Epoch 3/12
  Training Loss: 1.3946611881256104
  Validation Loss: 3.400302921022688, Accuracy: 14.75%
Epoch 4/12
  Training Loss: 0.6792673468589783
  Validation Loss: 2.07527300289699, Accuracy: 43.25%
Epoch 5/12
  Training Loss: 0.35824859142303467
  Validation Loss: 1.842062967164176, Accuracy: 52.75%
Epoch 6/12
  Training Loss: 0.22821924090385437
  Validation Loss: 2.162138887814113, Accuracy: 45.25%
Epoch 7/12
  Training Loss: 0.24761153757572174
  Validation Loss: 2.45637880052839, Accuracy: 45.00%
Epoch 8/12
  Training Loss: 0.06005482375621796
  Validation Loss: 2.783326659883772, Accuracy: 34.00%
Epoch 9/12
  Training Loss: 0.04300304129719734
  Validation Loss: 0.9165707230567932, Accuracy: 77.50%
Epoch 10/12
  Training Loss: 0.015345900319516659
  Validation Loss: 0.89871859

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')
resnet50.load_state_dict(checkpoint)

generate_adversarial(esc, resnet50, 'resnet50')

all attacks loaded
dataset for fgsm generated
fgsm adv dataset for resnet50 stored, time 18.840731620788574
dataset for bim generated
bim adv dataset for resnet50 stored, time 120.97313332557678
dataset for pgd generated
pgd adv dataset for resnet50 stored, time 122.3922438621521
dataset for cw generated
cw adv dataset for resnet50 stored, time 513.4668953418732


In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')
resnet50.load_state_dict(checkpoint)

for_all(resnet50, 'resnet50')

In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'resnet50'


    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/ESC50/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)

        resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
        resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet50.fc = nn.Linear(in_features=2048, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')
        resnet50.load_state_dict(checkpoint)
        r18 = ret_pred(resnet50, dataset_class)

        print(f'Ratio for {adv_dir}/{attk} dataset')

        resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
        resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet18.fc = nn.Linear(in_features=512, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet18.pth')
        resnet18.load_state_dict(checkpoint)
        r50 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
        vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        vit_base.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')
        vit_base.load_state_dict(checkpoint)
        vb = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
        mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        mixer.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/mixer.pth')
        mixer.load_state_dict(checkpoint)
        m = ret_pred(mixer, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        del dataset_class
        del data_list
        del label_list

cal_ratio()

Ratio for resnet50/fgsm dataset
0.5805 0.1 0.3195
0.7945 0.022 0.1835
0.922 0.0115 0.0665
Ratio for resnet50/bim dataset
0.8565 0.044 0.0995
0.802 0.027 0.171
0.924 0.015 0.061
Ratio for resnet50/pgd dataset
0.867 0.056 0.077


# VIT Base

In [ ]:
import timm

vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
vit_base.head = nn.Linear(in_features=768, out_features=50, bias=True)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(vit_base, esc, learning_rate=1e-4, num_epochs=15, batch_size=32)

torch.save(vit_base.state_dict(), '/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')

Epoch 1/15
  Training Loss: 3.7263689041137695
  Validation Loss: 3.8602323898902307, Accuracy: 4.25%
Epoch 2/15
  Training Loss: 3.9138424396514893
  Validation Loss: 3.5325343975654016, Accuracy: 7.00%
Epoch 3/15
  Training Loss: 3.633577346801758
  Validation Loss: 3.4331500897040734, Accuracy: 9.50%
Epoch 4/15
  Training Loss: 3.414691925048828
  Validation Loss: 3.1967229659740743, Accuracy: 12.25%
Epoch 5/15
  Training Loss: 3.2346670627593994
  Validation Loss: 3.061866411795983, Accuracy: 14.25%
Epoch 6/15
  Training Loss: 2.7240655422210693
  Validation Loss: 2.9107314073122463, Accuracy: 17.50%
Epoch 7/15
  Training Loss: 2.619856119155884
  Validation Loss: 2.7722511291503906, Accuracy: 20.50%
Epoch 8/15
  Training Loss: 2.3691608905792236
  Validation Loss: 2.427374638043917, Accuracy: 30.75%
Epoch 9/15
  Training Loss: 2.1953160762786865
  Validation Loss: 2.4257663029890795, Accuracy: 30.25%
Epoch 10/15
  Training Loss: 1.9832791090011597
  Validation Loss: 2.131519730274

### *Adversarial data*

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')
vit_base.load_state_dict(checkpoint)

generate_adversarial(esc, vit_base, 'vit_base')

all attacks loaded
dataset for fgsm generated
fgsm adv dataset for vit_base stored, time 49.910452365875244
dataset for bim generated
bim adv dataset for vit_base stored, time 481.8449945449829
dataset for pgd generated
pgd adv dataset for vit_base stored, time 481.7501857280731
dataset for cw generated
cw adv dataset for vit_base stored, time 2878.0130088329315


In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')
vit_base.load_state_dict(checkpoint)

for_all(vit_base, 'vit_base')

Stats for vit_base on resnet18/fgsm datasetAccuracy: 79.65%
Stats for vit_base on resnet18/bim datasetAccuracy: 79.65%
Stats for vit_base on resnet18/pgd datasetAccuracy: 79.85%
Stats for vit_base on resnet18/cw datasetAccuracy: 80.30%
Stats for vit_base on resnet50/fgsm datasetAccuracy: 79.45%
Stats for vit_base on resnet50/bim datasetAccuracy: 80.20%
Stats for vit_base on resnet50/pgd datasetAccuracy: 80.15%
Stats for vit_base on resnet50/cw datasetAccuracy: 80.20%
Stats for vit_base on vit_base/fgsm datasetAccuracy: 2.80%
Stats for vit_base on vit_base/bim datasetAccuracy: 0.00%
Stats for vit_base on vit_base/pgd datasetAccuracy: 0.15%
Stats for vit_base on vit_base/cw datasetAccuracy: 1.55%
Stats for vit_base on vit_large/fgsm datasetAccuracy: 64.55%
Stats for vit_base on vit_large/bim datasetAccuracy: 65.10%
Stats for vit_base on vit_large/pgd datasetAccuracy: 68.75%
Stats for vit_base on mixer/fgsm datasetAccuracy: 78.20%
Stats for vit_base on mixer/bim datasetAccuracy: 79.00%
St

In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'vit_base'


    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/ESC50/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)



        vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
        vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        vit_base.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')
        vit_base.load_state_dict(checkpoint)
        r18 = ret_pred(vit_base, dataset_class)

        print(f'Ratio for {adv_dir}/{attk} dataset')

        resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
        resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet18.fc = nn.Linear(in_features=512, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet18.pth')
        resnet18.load_state_dict(checkpoint)
        r50 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
        resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet50.fc = nn.Linear(in_features=2048, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')
        resnet50.load_state_dict(checkpoint)
        vb = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
        mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        mixer.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/mixer.pth')
        mixer.load_state_dict(checkpoint)
        m = ret_pred(mixer, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        del dataset_class
        del data_list
        del label_list

cal_ratio()

# VIT - Large

In [ ]:
import timm

vit_large = timm.create_model('vit_large_patch16_224.augreg_in21k_ft_in1k', pretrained=True)
vit_large.patch_embed.proj = nn.Conv2d(1, 1024, kernel_size=(16, 16), stride=(16, 16))
vit_large.head = nn.Linear(in_features=1024, out_features=50, bias=True)

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(vit_large, esc, learning_rate=1e-4, num_epochs=12, batch_size=32)

torch.save(vit_large.state_dict(), '/content/drive/MyDrive/data/Models/ESC50/vit_large.pth')

Epoch 1/20
  Training Loss: 3.915684938430786
  Validation Loss: 3.7950846598698544, Accuracy: 2.00%
Epoch 2/20
  Training Loss: 3.6159818172454834
  Validation Loss: 3.616451483506423, Accuracy: 5.25%
Epoch 3/20
  Training Loss: 3.7879998683929443
  Validation Loss: 3.2852648955125074, Accuracy: 9.50%
Epoch 4/20
  Training Loss: 3.1280300617218018
  Validation Loss: 3.1906901873075046, Accuracy: 13.25%
Epoch 5/20
  Training Loss: 2.789907932281494
  Validation Loss: 2.8750778895158033, Accuracy: 20.25%
Epoch 6/20
  Training Loss: 2.127769947052002
  Validation Loss: 2.5861577895971446, Accuracy: 25.50%
Epoch 7/20
  Training Loss: 1.8549036979675293
  Validation Loss: 2.1194185477036696, Accuracy: 37.50%
Epoch 8/20
  Training Loss: 1.6993825435638428
  Validation Loss: 2.008920036829435, Accuracy: 41.50%
Epoch 9/20
  Training Loss: 2.093467950820923
  Validation Loss: 1.777286749619704, Accuracy: 45.25%
Epoch 10/20
  Training Loss: 1.103757619857788
  Validation Loss: 1.653772748433626

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_large.pth')
vit_large.load_state_dict(checkpoint)

gen_cw(esc, vit_large, 'vit_large')

all attacks loaded


In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_large.pth')
vit_large.load_state_dict(checkpoint)

for_all(vit_large, 'vit_large')

Stats for vit_large on resnet18/fgsm datasetAccuracy: 92.85%
Stats for vit_large on resnet18/bim datasetAccuracy: 92.80%
Stats for vit_large on resnet18/pgd datasetAccuracy: 92.85%
Stats for vit_large on resnet18/cw datasetAccuracy: 92.90%
Stats for vit_large on resnet50/fgsm datasetAccuracy: 92.80%
Stats for vit_large on resnet50/bim datasetAccuracy: 92.90%
Stats for vit_large on resnet50/pgd datasetAccuracy: 93.00%
Stats for vit_large on resnet50/cw datasetAccuracy: 92.90%
Stats for vit_large on vit_base/fgsm datasetAccuracy: 86.25%
Stats for vit_large on vit_base/bim datasetAccuracy: 86.50%
Stats for vit_large on vit_base/pgd datasetAccuracy: 88.50%
Stats for vit_large on vit_base/cw datasetAccuracy: 92.65%
Stats for vit_large on vit_large/fgsm datasetAccuracy: 6.55%
Stats for vit_large on vit_large/bim datasetAccuracy: 0.40%
Stats for vit_large on vit_large/pgd datasetAccuracy: 0.95%
Stats for vit_large on mixer/fgsm datasetAccuracy: 91.90%
Stats for vit_large on mixer/bim datasetA

# MLP Mixer

In [ ]:
import timm
mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
mixer.head = nn.Linear(in_features=768, out_features=50, bias=True)

model.safetensors:   0%|          | 0.00/240M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
train_model(mixer, esc, learning_rate=1e-4, num_epochs=12, batch_size=64)

torch.save(mixer.state_dict(), '/content/drive/MyDrive/data/Models/ESC50/mixer.pth')

Epoch 1/12
  Training Loss: 3.4929420948028564
  Validation Loss: 3.2465365954807828, Accuracy: 11.75%
Epoch 2/12
  Training Loss: 2.700716018676758
  Validation Loss: 2.6935834884643555, Accuracy: 27.75%
Epoch 3/12
  Training Loss: 1.9207773208618164
  Validation Loss: 2.1522699424198697, Accuracy: 42.75%
Epoch 4/12
  Training Loss: 1.2815324068069458
  Validation Loss: 2.025278057370867, Accuracy: 42.25%
Epoch 5/12
  Training Loss: 1.1761680841445923
  Validation Loss: 1.6526193107877458, Accuracy: 53.25%
Epoch 6/12
  Training Loss: 0.41373518109321594
  Validation Loss: 1.4924282687050956, Accuracy: 58.50%
Epoch 7/12
  Training Loss: 0.119568832218647
  Validation Loss: 1.495565584727696, Accuracy: 58.25%
Epoch 8/12
  Training Loss: 0.07722233980894089
  Validation Loss: 1.4304983615875244, Accuracy: 60.25%
Epoch 9/12
  Training Loss: 0.055324483662843704
  Validation Loss: 1.3903829540525163, Accuracy: 62.00%
Epoch 10/12
  Training Loss: 0.03539208695292473
  Validation Loss: 1.400

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/mixer.pth')
mixer.load_state_dict(checkpoint)

generate_adversarial(esc, mixer, 'mixer')

all attacks loaded
dataset for fgsm generated
fgsm adv dataset for mixer stored, time 42.04572606086731
dataset for bim generated
bim adv dataset for mixer stored, time 359.32314682006836
dataset for pgd generated
pgd adv dataset for mixer stored, time 359.1354298591614
dataset for cw generated
cw adv dataset for mixer stored, time 1694.061113357544


In [ ]:
import timm

def ret_pred(model, dataset):
    data_loader = DataLoader(dataset, batch_size=128)
    model = model.to(device)
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for batch_input, batch_target in data_loader:
            batch_input = batch_input.to(device)
            predictions = model(batch_input)
            all_predictions.append(predictions.argmax(dim=1))

    all_predictions = torch.cat(all_predictions)
    return all_predictions

def cal_ratio():
    attk_list = ['fgsm', 'bim', 'pgd', 'cw']
    adv_dir = 'mixer'


    for attk in attk_list:
        h5_file_path = f'/content/drive/MyDrive/data/AdvData/ESC50/{adv_dir}/{attk}.h5'
        data_list = []
        label_list = []

        with h5py.File(h5_file_path, 'r') as hf:
            # Iterate over the samples in the HDF5 file
            for sample_name in hf.keys():
                data = torch.tensor(hf[sample_name]['data'][:])
                label = hf[sample_name]['label'][()]

                # Append data and label to lists
                data_list.append(data)
                label_list.append(label)

        dataset_class = TensorDataset(torch.stack(data_list), torch.tensor(label_list))
        label_list = torch.tensor(label_list)


        mixer = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
        mixer.stem.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        mixer.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/mixer.pth')
        mixer.load_state_dict(checkpoint)
        r18 = ret_pred(mixer, dataset_class)

        print(f'Ratio for {adv_dir}/{attk} dataset')

        resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
        resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet18.fc = nn.Linear(in_features=512, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet18.pth')
        resnet18.load_state_dict(checkpoint)
        r50 = ret_pred(resnet18, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (r50[i] == label_list[i]):
                unfooled += 1
            elif (r50[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
        resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        resnet50.fc = nn.Linear(in_features=2048, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/resnet50.pth')
        resnet50.load_state_dict(checkpoint)
        vb = ret_pred(resnet50, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (vb[i] == label_list[i]):
                unfooled += 1
            elif (vb[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
        vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
        vit_base.head = nn.Linear(in_features=768, out_features=50, bias=True)
        torch.cuda.empty_cache()
        checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/vit_base.pth')
        vit_base.load_state_dict(checkpoint)
        m = ret_pred(vit_base, dataset_class)

        unfooled = 0
        diff = 0
        same = 0
        for i in range(2000):
            if (m[i] == label_list[i]):
                unfooled += 1
            elif (m[i] == r18[i]):
                same += 1
            else:
                diff += 1

        print (unfooled/2000, same/2000, diff/2000)

        del dataset_class
        del data_list
        del label_list

cal_ratio()

# OTHER Datasets


In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/ESC50/mixer.pth')
mixer.load_state_dict(checkpoint)

for_all(mixer, 'mixer')

Stats for mixer on resnet18/fgsm datasetAccuracy: 92.10%
Stats for mixer on resnet18/bim datasetAccuracy: 92.55%
Stats for mixer on resnet18/pgd datasetAccuracy: 92.45%
Stats for mixer on resnet18/cw datasetAccuracy: 92.35%
Stats for mixer on resnet50/fgsm datasetAccuracy: 92.20%
Stats for mixer on resnet50/bim datasetAccuracy: 92.40%
Stats for mixer on resnet50/pgd datasetAccuracy: 92.10%
Stats for mixer on resnet50/cw datasetAccuracy: 92.30%
Stats for mixer on vit_base/fgsm datasetAccuracy: 91.10%
Stats for mixer on vit_base/bim datasetAccuracy: 90.95%
Stats for mixer on vit_base/pgd datasetAccuracy: 91.05%
Stats for mixer on vit_base/cw datasetAccuracy: 92.40%
Stats for mixer on vit_large/fgsm datasetAccuracy: 88.30%
Stats for mixer on vit_large/bim datasetAccuracy: 87.90%
Stats for mixer on vit_large/pgd datasetAccuracy: 89.65%
Stats for mixer on mixer/fgsm datasetAccuracy: 2.90%
Stats for mixer on mixer/bim datasetAccuracy: 0.00%
Stats for mixer on mixer/pgd datasetAccuracy: 0.00%

In [ ]:
resnet18 = timm.create_model('resnet18.a1_in1k', pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=27, bias=True)

torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/FSC22/resnet18.pth')
resnet18.load_state_dict(checkpoint)

for_all(resnet18, 'resnet18')

Stats for resnet18 on resnet18/fgsm datasetAccuracy: 12.40%
Stats for resnet18 on resnet18/bim datasetAccuracy: 0.00%
Stats for resnet18 on resnet18/pgd datasetAccuracy: 0.00%
Stats for resnet18 on resnet18/cw datasetAccuracy: 0.00%
Stats for resnet18 on resnet50/fgsm datasetAccuracy: 62.12%
Stats for resnet18 on resnet50/bim datasetAccuracy: 85.78%
Stats for resnet18 on resnet50/pgd datasetAccuracy: 86.17%
Stats for resnet18 on resnet50/cw datasetAccuracy: 94.27%
Stats for resnet18 on vit_base/fgsm datasetAccuracy: 69.98%
Stats for resnet18 on vit_base/bim datasetAccuracy: 76.10%
Stats for resnet18 on vit_base/pgd datasetAccuracy: 81.78%
Stats for resnet18 on vit_base/cw datasetAccuracy: 94.27%
Stats for resnet18 on vit_large/fgsm datasetAccuracy: 62.86%
Stats for resnet18 on vit_large/bim datasetAccuracy: 83.31%
Stats for resnet18 on vit_large/pgd datasetAccuracy: 85.63%
Stats for resnet18 on mixer/fgsm datasetAccuracy: 64.20%
Stats for resnet18 on mixer/bim datasetAccuracy: 84.05%
S

In [ ]:
resnet50 = timm.create_model('resnet50.a1_in1k', pretrained=True)
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet50.fc = nn.Linear(in_features=2048, out_features=27, bias=True)

torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/FSC22/resnet50.pth')
resnet50.load_state_dict(checkpoint)

for_all(resnet50, 'resnet50')

Stats for resnet50 on resnet18/fgsm datasetAccuracy: 72.49%
Stats for resnet50 on resnet18/bim datasetAccuracy: 88.25%
Stats for resnet50 on resnet18/pgd datasetAccuracy: 89.38%
Stats for resnet50 on resnet18/cw datasetAccuracy: 94.91%
Stats for resnet50 on resnet50/fgsm datasetAccuracy: 23.85%
Stats for resnet50 on resnet50/bim datasetAccuracy: 0.00%
Stats for resnet50 on resnet50/pgd datasetAccuracy: 0.00%
Stats for resnet50 on resnet50/cw datasetAccuracy: 0.10%
Stats for resnet50 on vit_base/fgsm datasetAccuracy: 76.84%
Stats for resnet50 on vit_base/bim datasetAccuracy: 81.33%
Stats for resnet50 on vit_base/pgd datasetAccuracy: 86.42%
Stats for resnet50 on vit_base/cw datasetAccuracy: 94.96%
Stats for resnet50 on vit_large/fgsm datasetAccuracy: 75.21%
Stats for resnet50 on vit_large/bim datasetAccuracy: 89.19%
Stats for resnet50 on vit_large/pgd datasetAccuracy: 90.27%
Stats for resnet50 on mixer/fgsm datasetAccuracy: 75.01%
Stats for resnet50 on mixer/bim datasetAccuracy: 89.09%
S

In [ ]:
torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/FSC22/mixer.pth')
mixer.load_state_dict(checkpoint)

for_all(mixer, 'mixer')

Stats for mixer on resnet18/fgsm datasetAccuracy: 92.84%
Stats for mixer on resnet18/bim datasetAccuracy: 92.79%
Stats for mixer on resnet18/pgd datasetAccuracy: 92.79%
Stats for mixer on resnet18/cw datasetAccuracy: 92.79%
Stats for mixer on resnet50/fgsm datasetAccuracy: 92.64%
Stats for mixer on resnet50/bim datasetAccuracy: 92.74%
Stats for mixer on resnet50/pgd datasetAccuracy: 92.59%
Stats for mixer on resnet50/cw datasetAccuracy: 92.84%
Stats for mixer on vit_base/fgsm datasetAccuracy: 92.05%
Stats for mixer on vit_base/bim datasetAccuracy: 91.90%
Stats for mixer on vit_base/pgd datasetAccuracy: 92.05%
Stats for mixer on vit_base/cw datasetAccuracy: 92.79%
Stats for mixer on vit_large/fgsm datasetAccuracy: 92.35%
Stats for mixer on vit_large/bim datasetAccuracy: 92.49%
Stats for mixer on vit_large/pgd datasetAccuracy: 92.69%
Stats for mixer on mixer/fgsm datasetAccuracy: 1.38%
Stats for mixer on mixer/bim datasetAccuracy: 0.00%
Stats for mixer on mixer/pgd datasetAccuracy: 0.00%

In [ ]:
vit_base = timm.create_model('vit_base_patch16_224.augreg2_in21k_ft_in1k', pretrained=True)
vit_base.patch_embed.proj = nn.Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
vit_base.head = nn.Linear(in_features=768, out_features=27, bias=True)

torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/FSC22/vit_base.pth')
vit_base.load_state_dict(checkpoint)

for_all(vit_base, 'vit_base')

Stats for vit_base on resnet18/fgsm datasetAccuracy: 68.25%
Stats for vit_base on resnet18/bim datasetAccuracy: 68.54%
Stats for vit_base on resnet18/pgd datasetAccuracy: 68.74%
Stats for vit_base on resnet18/cw datasetAccuracy: 68.64%
Stats for vit_base on resnet50/fgsm datasetAccuracy: 68.30%
Stats for vit_base on resnet50/bim datasetAccuracy: 68.30%
Stats for vit_base on resnet50/pgd datasetAccuracy: 68.44%
Stats for vit_base on resnet50/cw datasetAccuracy: 68.84%
Stats for vit_base on vit_base/fgsm datasetAccuracy: 4.40%
Stats for vit_base on vit_base/bim datasetAccuracy: 0.05%
Stats for vit_base on vit_base/pgd datasetAccuracy: 0.40%
Stats for vit_base on vit_base/cw datasetAccuracy: 4.54%
Stats for vit_base on vit_large/fgsm datasetAccuracy: 66.91%
Stats for vit_base on vit_large/bim datasetAccuracy: 67.70%
Stats for vit_base on vit_large/pgd datasetAccuracy: 67.51%
Stats for vit_base on mixer/fgsm datasetAccuracy: 66.57%
Stats for vit_base on mixer/bim datasetAccuracy: 67.85%
St

In [ ]:
import timm
vit_large = timm.create_model('vit_large_patch16_224.augreg_in21k_ft_in1k', pretrained=True)
vit_large.patch_embed.proj = nn.Conv2d(1, 1024, kernel_size=(16, 16), stride=(16, 16))
vit_large.head = nn.Linear(in_features=1024, out_features=27, bias=True)

torch.cuda.empty_cache()
checkpoint = torch.load('/content/drive/MyDrive/data/Models/FSC22/vit_large.pth', map_location=torch.device(device))
vit_large.load_state_dict(checkpoint)

for_all(vit_large, 'vit_large')

Stats for vit_large on resnet18/fgsm dataset
Accuracy: 92.20%
Stats for vit_large on resnet18/bim dataset
Accuracy: 92.40%
Stats for vit_large on resnet18/pgd dataset
Accuracy: 92.49%
Stats for vit_large on resnet18/cw dataset
Accuracy: 92.40%
Stats for vit_large on resnet50/fgsm dataset
Accuracy: 92.25%
Stats for vit_large on resnet50/bim dataset
Accuracy: 92.49%
Stats for vit_large on resnet50/pgd dataset
Accuracy: 92.35%
Stats for vit_large on resnet50/cw dataset
Accuracy: 92.35%
Stats for vit_large on vit_base/fgsm dataset
Accuracy: 90.77%
Stats for vit_large on vit_base/bim dataset
Accuracy: 91.11%
Stats for vit_large on vit_base/pgd dataset
Accuracy: 91.46%
Stats for vit_large on vit_base/cw dataset
Accuracy: 92.40%
Stats for vit_large on vit_large/fgsm dataset
Accuracy: 6.27%
Stats for vit_large on vit_large/bim dataset
Accuracy: 0.00%
Stats for vit_large on vit_large/pgd dataset
Accuracy: 0.00%
Stats for vit_large on mixer/fgsm dataset
Accuracy: 92.20%
Stats for vit_large on mi